# Модерация текстов
<b>Задача:</b> Создать модель, которая будет определять эмоциональную окраску текста и пропускать его или же отправлять на модерацию. Построить модель со значением метрики качества F1 не меньше 0.75.<br>
<b>Дано:</b> Набор текстов с разметкой их эмоциональной окраски. На основе этих данных модель должна научиться классифицировать текст как позитивный или негативный.

## Подготовка данных

In [ ]:
!pip install catboost -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 10.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
from nltk.stem import WordNetLemmatizer
import re
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from tqdm import tqdm
tqdm.pandas()
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from catboost import CatBoostClassifier, Pool, cv

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

corpus = data['text']

print('Доля негативных твитов в датасете:', round(data['toxic'].sum() / len(data['toxic']), 2))

Доля негативных твитов в датасете: 0.1


In [ ]:
nlp = spacy.load('en_core_web_sm')

stopwords = list(stopwords.words('english'))

def clear_text(text):
    text_sub = re.sub(r'[^a-zA-Z ]', ' ', text)
    return ' '.join(text_sub.split())

In [ ]:
corpus_lemm = []

for doc in tqdm(nlp.pipe(corpus.apply(clear_text), batch_size=64, n_process=-1, disable=["parser", "ner"]), total=len(corpus)):
    word_list = [tok.lemma_ for tok in doc]
    corpus_lemm.append(' '.join(word_list))

100%|██████████| 159292/159292 [16:51<00:00, 157.49it/s]


### Вывод
Провели первичную подготовку данных. Тексты твитов выгружены в корпус, проведена лемматизация и убраны стоп-слова.

## Векторизация текстов


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    corpus_lemm, data['toxic'], test_size=0.3, random_state=12345, stratify=data['toxic'])

### Вывод
Проведено разбитие выборки на тренировочную и тестовую с учетом стратификации по целевому признаку, далее выборки были преобразованы в векторы TF-IDF для дальнейшей передачи в модель.

## Обучение моделей

### Дамми модель

In [ ]:
predictions = pd.Series(0, index=np.arange(len(y_test)))
print('F1-мера для дамми модели =', f1_score(y_test, predictions, zero_division=1))
print('Accuracy для дамми модели =', accuracy_score(y_test, predictions))

F1-мера для дамми модели = 0.0
Accuracy для дамми модели = 0.8983845316815937


Дамми модель, заполненная нулями (т.к. 90% датасета - это комментарии положительного эмоционального окраса), выдает ожидаемую accuracy ~89%, но при этом F1-мера недостаточна джля дальнейшего сравнения с этой моделью.

### LogisticRegression

In [ ]:
%%time
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('logreg', LogisticRegression(random_state=42, max_iter=500)),
])
parameters = {'logreg__C' : [1e-2, 1e-1, 1, 10, 30],
              'logreg__penalty': ['l1', 'l2', 'elasticnet']}

lr = GridSearchCV(pipeline, parameters, scoring='f1', cv=3, n_jobs=1)
scores_lr = lr.fit(X_train, y_train)
print('Лучшая F1-мера для LogisticRegression', round(scores_lr.best_score_, 3))
print(scores_lr.best_params_)

/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
30 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  Fil

Лучшая F1-мера для LogisticRegression 0.768
{'logreg__C': 10, 'logreg__penalty': 'l2'}
CPU times: user 6min 9s, sys: 2min 37s, total: 8min 47s
Wall time: 6min 48s


### LinearSVC

In [ ]:
%%time
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('lsvc', LinearSVC(max_iter = 500)),
])
parameters = {'lsvc__C': np.linspace(1, 31, num = 7, endpoint = True)}

model_lsvc = GridSearchCV(pipeline, parameters, cv=5, scoring='f1', n_jobs=-1, verbose=2)
scores_lsvc  = model_lsvc.fit(X_train, y_train)
print('Лучшая F1-мера для LinearSVC', round(scores_lsvc.best_score_, 3))
print(scores_lsvc.best_params_)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
Лучшая F1-мера для LinearSVC 0.778
{'lsvc__C': 1.0}
CPU times: user 12.4 s, sys: 3.29 s, total: 15.7 s
Wall time: 5min 20s


### DecisionTreeClassifier

In [ ]:
%%time
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('dtc', DecisionTreeClassifier(random_state=12345)),
])
parameters =  {
    'dtc__min_samples_leaf': [1, 2],
    'dtc__max_depth': [6, 30, 100]
}

model_dtc = GridSearchCV(pipeline, parameters, cv=5, scoring='f1', n_jobs=-1, verbose=2)
scores_dtc  = model_dtc.fit(X_train, y_train)
print('Лучшая F1-мера для DecisionTreeClassifier', round(scores_dtc.best_score_, 3))
print(scores_dtc.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Лучшая F1-мера для DecisionTreeClassifier 0.727
{'dtc__max_depth': 100, 'dtc__min_samples_leaf': 1}
CPU times: user 1min 20s, sys: 2.94 s, total: 1min 23s
Wall time: 12min 58s


### Вывод
Были проверены разные модели классификации с подбором гиперпараметров. Лучшие результаты показала модель решающего дерева, но вероятно эта цифра может говорить о переобчуении. Однако, она обучалась дольше всех других моделей, около 1 минуты после подбора гиперпараметров, что заняло 20 минут. Если для задачи важно время обучения модели, то стоит рекомендовать модель опорных векторов, обучение модели происходит меньбше, чем за секунду, показатели F1-меры удовлетворяют условию задачи. Эту модель можно передать на тестирование.


## Тестирование модели

In [ ]:
predictions = model_lsvc.predict(X_test)
print('F1-мера на тестовой выборке для LinearSVC модели', round(f1_score(predictions, y_test), 2))

F1-мера на тестовой выборке для LinearSVC модели 0.78


### Вывод
Модель опорных векторов прошла тестирование и показала удовлетворительный результат по заказанной метрике. Моджель можно рекомендовать заказчику.

## Общий вывод
Была проведена работа по построению модели для анализа эмоциональной окраски поступающего текста и определения его как допустимого или же требующего модерации. Входными данными был датасет с размеченными текстами, по этим данным была создана модель, отвечающая поставленной задаче. Заказчику рекомендована модель LinearSVC (C=1), которая очень быстра в работе и выдает хороший результат по заданной метрике (F1-мера) на тесте.